In [5]:
%load_ext autoreload
%autoreload 2

In [6]:
import numpy as np
import functools as ft

def less_than(a, b):
    '''
    Assuming a and b are on adjacent vertices of
    a z-order curve, return -1 if a < b, 0 if
    a == b and 1 if a > b
    
    Parameters
    ----------
    a: (float, float)
        Point
    b: (float, float)
        Point
    '''
    
    x = np.array(a)
    y = np.array(b)
        
    ret = 1
    
    if a[0] == b[0] and a[1] == b[1]:
        return 0
    
    if a[1] < b[1]:
        ret = -1
    else:
        if a[0] < b[0]:
            ret = -1
    if a[1] > b[1]:
        ret = 1
    
    #print(ret)
    return ret
    
    print(a[1] < b[1], a[0] < b[0], a[1] > b[1], ret)
    #return True if a[1] <= b[1] else a[0] <= b[0]
    #print(a[1] <= b[1], a[0] <= b[0]
    
less_than([0,0], [0,1]) # -1 or True
less_than([0,0], [1,0]) # -1 or True
less_than([0,0], [1,1]) # -1 or True

less_than([1,0], [0,1]) # -1 or True
less_than([1,0], [1,1]) # -1 or True

less_than([0,1], [1,1]) # -1 or True
print("-------------")

less_than([0,1], [0,0]) #
less_than([1,0], [0,0]) #
less_than([1,1], [0,0]) #

less_than([0,1], [1,0]) # False
less_than([1,1], [1,0]) #

less_than([1,1], [0,1]) #

-------------


1

In [81]:
def in_tile(a, tile):
    '''
    Is a point in a tile
    
    Parameters
    ----------
    a: (float, float)
    b: (float, float)
    tile: (float, float, float, float)
    '''
    return a[0] >= tile[0] and a[1] >= tile[1] and a[0] < tile[2] and a[1] < tile[3]

print(in_tile([1,1], [0,0,1,1]))
print(in_tile([2,2], [0,0,1,1]))
print(in_tile([-1,1], [0,0,1,1]))
print(in_tile([0.5,0.5], [0,0,1,1]))

False
False
False
True


In [82]:
def tile_children(tile):
    '''
    Split a tile into its four children
    
    Parameters
    ----------
    tile: (float, float, float, float)
    '''
    tile_width = tile[2] - tile[0]
    tile_height = tile[3] - tile[1]
    
    return [
        [tile[0], tile[1], tile[0] + tile_width / 2, tile[1] + tile_height / 2],
        [tile[0], tile[1] + tile_height / 2, tile[0] + tile_width / 2, tile[3]],
        [tile[0] + tile_width / 2, tile[1], tile[2], tile[1] + tile_height / 2],
        [tile[0] + tile_width / 2, tile[1] + tile_height / 2, tile[2], tile[3]]
    ]
        
tile_children([0,0,1,1])

[[0, 0, 0.5, 0.5], [0, 0.5, 0.5, 1], [0.5, 0, 1, 0.5], [0.5, 0.5, 1, 1]]

In [83]:
big_tile=[0,0,8,8]

In [113]:
def zindex_compare(a,b,tile):
    '''
    Compare two points along z curve wihin the bounds of 
    the given tile.
    
    Parameters
    ----------
    a: (float, float)
        Point 1
    b: (float, float)
        Point 2
    tile: (float, float, float, float)
        The bounds of the region in which we want to compare them
    '''
    #print(a, tile, in_tile(a,tile))
    #print("a,b,tile", a,b, tile)
    assert(in_tile(a,tile))
    assert(in_tile(b, tile))
    
    #print("comparing", a, b)
    
    if a[0] == b[0] and a[1] == b[1]:
        return 0
    
    children = tile_children(tile)
    
    for child in children:
        #print("child:", child)
        
        if in_tile(a, child):
            child_a = child
        if in_tile(b, child):
            child_b = child
            
    if child_a == child_b:
        #print("child_a", child_a, "child_b", child_b)
        return zindex_compare(a, b, child_a)
    else:
        return less_than(child_a[:2], child_b[:2])
        

assert(zindex_compare([2,2], [2,2], [0,0,8,8]) == 0)
assert(zindex_compare([0,4], [3,4], [0,0,8,8]) == -1)
assert(zindex_compare([6,4], [3,1], [0,0,8,8]) == 1)
assert(zindex_compare([3,1], [2,2], [0,0,8,8]) == -1)
assert(zindex_compare([2,2], [3,1], [0,0,8,8]) == 1)
assert(zindex_compare([7,1], [2,2], [0,0,8,8]) == 1)
assert(zindex_compare([2,2], [7,1], [0,0,8,8]) == -1)

In [114]:
%%time

points_list = sorted([
    [6,4],[7,1],[2,2],[3,1],[3,4],[0,4],[1,6],[4,2]
], key=ft.cmp_to_key(lambda a,b: zindex_compare(a,b, [0,0,8,8])))

print(points_list)

[[3, 1], [2, 2], [7, 1], [4, 2], [0, 4], [3, 4], [1, 6], [6, 4]]
CPU times: user 448 µs, sys: 206 µs, total: 654 µs
Wall time: 472 µs


In [115]:
def lower_bound(sequence, value, compare=None):
    """Find the index of the first element in sequence >= value"""
    elements = len(sequence)
    offset = 0
    middle = 0
    found = len(sequence)
 
    while elements > 0:
        middle = elements // 2
        #print("middle:", middle)
        if compare(value, sequence[offset + middle]) > 0:
            offset = offset + middle + 1
            elements = elements - (middle + 1)
        else:
            found = offset + middle
            elements = middle
    return found

def upper_bound(sequence, value, compare):
    """Find the index of the first element in sequence > value"""
    elements = len(sequence)
    offset = 0
    middle = 0
    found = 0
 
    while elements > 0:
        middle = elements // 2
        if compare(value, sequence[offset + middle]) < 0:
            elements = middle
        else:
            offset = offset + middle + 1
            found = offset
            elements = elements - (middle + 1)
    return found

print(lower_bound(points_list, [1,6], lambda a,b: zindex_compare(a,b, [0,0,8,8])))
print(upper_bound(points_list, [1,6], lambda a,b: zindex_compare(a,b, [0,0,8,8])))

6
7


In [230]:
def all_point_boundaries(points_list, tile, bounding_tile):
    cmp = lambda a,b: zindex_compare(a,b, bounding_tile)
    
    left_index = lower_bound(points_list, [tile[0], tile[1]], cmp)
    right_index = lower_bound(points_list, [tile[2] - 0.000001, tile[3] - 0.000001], cmp)
    #print("right_index", right_index)
    
    return left_index, right_index

def all_points(points_list, tile, bounding_tile):
    '''
    Return all points that are in the tile given tile
    '''
    left_index, right_index = all_point_boundaries(points_list, tile, bounding_tile)
    
    return points_list[left_index:right_index]

all_points(points_list, [0.0, 0.0, 4.0, 4.0], big_tile)

[[3, 1], [2, 2]]

In [231]:
#all_points(points_list, [3.0, 1.9999999999999998, 3.0000000000000004, 2.0]

In [234]:
def all_in(tile, rect):
    '''
    Is this tile completely enclosed in the rectangle:
    
    Parameters
    ----------
    tile: [float, float, float, float]
        The xmin,ymin,xmax,ymax of the tile
    rect: [float, float, float, float]
        The xmin, ymin, xmas, ymax of the rectangle
    '''
    ret = (tile[0] >= rect[0] and
            tile[1] >= rect[1] and 
            tile[2] <= rect[2] and
            tile[3] <= rect[3])
    #print("all_in:", ret, tile, rect)
    return ret

def some_in(tile, rect):
    '''
    Is part of this tile within the rectangle?
    
    Parameters
    ----------
    tile: [float, float, float, float]
        The xmin,ymin,xmax,ymax of the tile
    rect: [float, float, float, float]
        The xmin, ymin, xmas, ymax of the tile
    '''
    ret = (tile[0] < rect[2] and
            tile[1] < rect[3] and 
            tile[2] > rect[0] and
            tile[3] >= rect[1])
    
    #print("some_in", ret)
    return ret

def get_points(points_list, rect, tile, bounding_tile):
    '''
    Get all the points in the tile that intersect the rectangle
    
    Parameters
    ----------
    rect: [float, float, float, float]
        minx, miny, maxx, maxy
    tile: [float, float, float, float]
        minx, miny, maxx, maxy
    '''
    points = []
    
    left_index, right_index = all_point_boundaries(points_list, tile, bounding_tile)
    
    #if len(points_in_tile) == 0:
    #    return []
    
    #print("tile:", tile, points_in_tile)
    # points_in_tile
    if left_index == right_index:
        return []
    
    if not some_in(tile, rect):
        # no intersection
        return []
    if all_in(tile, rect):
        #print("all points", tile)
        return points_list[left_index:right_index]
    
    for child in tile_children(tile):
        points += get_points(points_list, rect, child, bounding_tile) 
        
    return points

In [235]:
get_points(points_list, [0,0,4,4], [0,0,8,8], [0,0,16,16])

[[3, 1], [2, 2]]

In [236]:
get_points(points_list, [0,0,5,5], [0,0,8,8], [0,0,16,16])

[[3, 1], [2, 2], [0, 4], [3, 4], [4, 2]]

In [237]:
get_points(points_list, [0,0,8,1.5], [0,0,8,8], [0,0,16,16])

[[3, 1], [7, 1]]

In [ ]:
%time new_points = np.random.random((64000,2))

test_bounds = [0,0,2,2]

%time new_points_list = sorted(new_points, key=ft.cmp_to_key(lambda a,b: zindex_compare(a,b, test_bounds)))

CPU times: user 3.7 ms, sys: 2.79 ms, total: 6.49 ms
Wall time: 13.1 ms


In [ ]:
def run_query_z_index():
    #print(query_bounds)
    min_pos = [random.random(), random.random()]
    a = 1 - min_pos[0]
    b = 1 - min_pos[1]
    max_pos = [min_pos[0] + a * random.random(), min_pos[1] + b * random.random()]
    query_bounds = min_pos + max_pos
    
    return get_points(new_points_list, query_bounds, test_bounds, test_bounds)

#def naive_filter():
    

%timeit run_query_z_index()
#print(len(x), query_bounds)

1000: 241ms
2000: 435ms
4000: 318ms
8000: 1.05s
16000: 2.4s
32000: 1.97s